In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import os.path
import datetime 

In [9]:
from surveysim.wrapper import surveySim

In [10]:
def collect_tile_id(start, end):
    day_now = start
    id_list = np.empty((0), dtype=int)
    id_plan = np.empty((0), dtype=int)
    while(day_now <= end):
        day_now = day_now + datetime.timedelta(days=1)
        listdirName = "{}{}{:02d}{:02d}{}".format('obslist',day_now.year, day_now.month, day_now.day, '.fits')   
        plandirName = "{}{}{:02d}{:02d}{}".format('obsplan',day_now.year, day_now.month, day_now.day, '.fits')    

        #print(day_now,listdirName)
        if os.path.exists(listdirName):
            a = Table.read(listdirName)
            set_a = set(a['TILEID'])
            set_id = set(id_list)
            set_diff = set_a - set_id
            id_list = np.append(id_list, np.array(list(set_diff), dtype=int)) #only add the IDs that don't repeat
        
        if os.path.exists(plandirName):
            a = Table.read(plandirName)
            set_a = set(a['TILEID'])
            set_id = set(id_plan)
            set_diff = set_a - set_id
            id_plan = np.append(id_plan, np.array(list(set_diff), dtype=int)) #only add the IDs that don't repeat
    
    return {'OBSERVED':id_list, 'PLANNED':id_plan}

In [11]:
! rm -f tiles_observed.fits
start_string = "01-01-2019"
start = datetime.datetime.strptime(start_string, "%d-%m-%Y")

n_epochs = 5
days_per_epoch = 4

fiberassign_cadence = datetime.timedelta(days=days_per_epoch)

end = start + fiberassign_cadence

for i in range(n_epochs):
    print('\n\n')
    surveySim((start.year, start.month, start.day), (end.year, end.month, end.day))
    dd = collect_tile_id(start, end)
    print dd['OBSERVED'], len(dd['OBSERVED']), len(dd['PLANNED'])
    print ('These were the observed tiles: {}'.format(dd['OBSERVED']))
    np.savetxt('observed_epoch{}.txt'.format(i), dd['OBSERVED'], fmt='%d')
    np.savetxt('epoch{}.txt'.format(i), dd['PLANNED'], fmt='%d')
    start = start + fiberassign_cadence 
    end = start + fiberassign_cadence




2019-01-01 19:00:00
The survey will start from scratch.
Conditions at the beginning of the night: 
{'Seeing': 0.9172205605527711, 'Clouds': 0.1526455474416512, 'Transparency': 0.6283410342193628, 'OpenDome': False}
('On the night starting ', '2019-01-01 19:00:00.000', ', we observed ', 0, ' tiles.')
Conditions at the beginning of the night: 
{'Seeing': 1.1802992427537113, 'Clouds': 0.21737847057836884, 'Transparency': 0.7681620117426027, 'OpenDome': False}
('On the night starting ', '2019-01-02 19:00:00.000', ', we observed ', 0, ' tiles.')
Conditions at the beginning of the night: 
{'Seeing': 0.9226219947307961, 'Clouds': 0.13816737671009177, 'Transparency': 0.3742492715595901, 'OpenDome': True}
('On the night starting ', '2019-01-03 19:00:00.000', ', we observed ', 15, ' tiles.')
Conditions at the beginning of the night: 
{'Seeing': 1.0837978472934415, 'Clouds': 0.14068275131852057, 'Transparency': 0.7538330976003925, 'OpenDome': True}
('On the night starting ', '2019-01-04 19:00:

In [18]:
full_tile_data = Table.read('/home/forero/desihub/surveysim/py/surveysim/data/desi-tiles.fits')
full_tile_data[3]

TILEID,RA,DEC,PASS,IN_DESI,EBV_MED,AIRMASS,EXPOSEFAC,STAR_DENSITY,PROGRAM,OBSCONDITIONS,I_INCREASE,G_INCREASE,HA,OBSTIME,OVERHEAD,BEGINOBS,ENDOBS,GALACLAT,GALACLON
,deg,deg,,,,,,1 / deg2,,,,,,,,,,,
int32,float32,float32,int16,int16,float32,float32,float32,float32,str4,uint16,float32,float32,float32,float32,float32,float32,float32,float64,float64
7,332.35,12.32,0,1,0.0744952,1.07357,1.0,3596.16,DARK,1,1.26239,1.57343,-9.10874,2000.95,180.0,21.2465,21.8523,-33.6823751944,72.7243045556


In [25]:
for i in range(n_epochs):
    id_plan = np.loadtxt("epoch{}.txt".format(i), dtype='int')
    id_list = np.loadtxt("observed_epoch{}.txt".format(i), dtype='int')
    print("epoch {}".format(i))
    print("planned {}".format(len(id_plan), id_plan))
    print("done {} {}\n\n".format(len(id_list), id_list))

done_tiles = Table.read("tiles_observed.fits")
print done_tiles
len(set(done_tiles['TILEID'])), set(done_tiles['STATUS']), 

epoch 0
planned 10092
done 38 [20866 26628 24453 24454 18684 20882 20883 24468 24118 24452 26683 20892
 20893 24446 18687 24449 24450 20875 20876 24469 24470 20891 20894 20895
 20896 20899 20901 24102 24634 24131 24132 24141 24653 24656 18390 17404
 24444 24447]


epoch 1
planned 10031
done 22 [24512 26656 18370 24484 26661 26663 24499 18891 18380 19170 18894 20911
 18896 18386 18387 24500 18872 26653 26654 26655 26664 24513]


epoch 2
planned 10009
done 18 [18722 19205 18927 24456 18697 24458 18892 18893 18895 18735 18738 18388
 24457 19192 18717 24475 24477 18718]


epoch 3
planned 9991
done 36 [18692 18694 18696 28779 18733 18734 18736 19154 28761 19198 18911 18690
 18691 18693 19206 18695 18704 18705 18706 18713 18715 18716 18723 18739
 18740 18747 18749 18750 18757 19158 18912 18913 18914 28780 18928 19193]


epoch 4
planned 9955
done 50 [18688 20871 17420 18703 18707 18708 18712 18714 18725 18732 18746 18748
 18751 18753 18759 19156 18915 19175 18416 18929 18930 18685 19199 19209

(187, {2})